In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [68]:
v = model.encode('I just discovered the course. Can I still join it?')

In [69]:
v[0]

0.07822264

In [8]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [12]:
ml_documents = []
for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':
        ml_documents.append(doc)

In [70]:
len(ml_documents)

375

In [71]:
ml_documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872',
 'question_text_vector': array([ 8.80591497e-02,  1.55936414e-02,  7.92557746e-02,  2.52758004e-02,
         7.55764544e-02, -3.90596539e-02, -4.13813330e-02,  2.52917204e-02,
         2.43242029e-02,  3.62590631e-03, -7.28289969e-03, -3.28750908e-02,
         6.12956323e-02, -5.71100861e-02,  1.16774756e-02, -1.79441366e-02,
         4.49205413e-02, -5.41606508e-02, -1.92254549e-03,  1.48329362e-02,
         7.91350473e-03, -3.43126729e-0

In [61]:
from tqdm.auto import tqdm

embeddings = []
for doc in tqdm(ml_documents):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'

    doc['question_text_vector'] = model.encode(qa_text)
    emb = model.encode(qa_text)
    embeddings.append(emb)
    

  0%|          | 0/375 [00:00<?, ?it/s]

In [72]:
import numpy as np
X = np.array(embeddings)
X.shape

(375, 768)

In [73]:
scores = X.dot(v)

In [74]:
max(scores)

0.6506574

In [75]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693',
  'question_text_vector': array([ 8.06286260e-02, -6.66388795e-02,  2.52730511e-02, -1.30044241e-02,
          7.58728385e-02, -5.94626553e-02, -2.18838584e-02,  2.90004467e-03,
          7.92880834e-04, -5.22236805e-03, -3.36518213e-02, -2.79135648e-02,
          5.81165552e-02,  3.97483148e-02,  5.44185601e-02, -3.82516421e-02,
          6.30573928e-02, -3.89698967e-02,  4.33601585e-04, -3.55845094e-02,
          1.75637053e-03

In [38]:
import pandas as pd

df_ground_truth = pd.read_csv('ground-truth-data.csv')
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [45]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [39]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [46]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    v = model.encode(q['question'])
    results =  search_engine.search(v, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [47]:
hit_rate(relevance_total)

0.9398907103825137

In [81]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [82]:
for doc in tqdm(ml_documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [85]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [87]:
elastic_search_knn('question_text_vector', v)

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

In [88]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    q = model.encode(q['question'])
    results =  elastic_search_knn('question_text_vector', q)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [89]:
hit_rate(relevance_total)

0.9398907103825137